## Brief introduction to R



### Interacting with the R Jupyter notebook

There are two types of Cells.
* "Markdown" cells allow you to write text like this one.
* The two cells below are "Code" cells. Type R commands into the cell and press shift-enter to compute the result.

In [ ]:
2*3

In [ ]:
10-3

The currently active Cell is indicated by a blue border and blue left margin. 

### R variables

A variable has a name and a value. For example, in the cell below we are assigning the value 2\*3 to the variable x. Press shift-enter to evaluate it.  Note that this command does not print the result on the screen (why?).

In [ ]:
x = 2*3

To view the content of the variable *x*, just type its name and press shift-enter.

In [ ]:
x

All variables in R are called *objects*. There are multiple types of objects in R, including scalars, vectors, matrices, arrays, data frames, tables, and lists.

A *vector* is a collection of *elements* of the same type.
We can use the `c()` function to create a vector with the name *myvector*.

In [ ]:
myvector = c(8,6,9,10,5)

Type the name of this vector to see its value.

In [ ]:
myvector

We can extract elements in the vector by its *index*. Here we are getting the first (position 1) and the fourth (position 4) element in the vector.

In [ ]:
myvector[1]

In [ ]:
myvector[4]

A *list* can contain elements of different types, for example, numbers or strings, or even vector. We can use the *list()* function to create a list. Values in the list can have names.

In [ ]:
mylist = list("USA"="Washington DC","Canada"="Ottawa",myvector)

To see the content of *mylist*, just type its name.

In [ ]:
mylist

We can get individual elements of the list by the name or the index using *double square brackets*.

In [ ]:
mylist[["Canada"]]

In [ ]:
mylist[[3]]

The `attributes` function gets all the names in the list.

### R functions
A *function* in R performs some calculations on input variables called *arguments* that are inside a round bracket. For example the `log10()` function calculates the base 10 log of the number passed in the round bracket.

In [ ]:
log10(100)

The `mean` function calculates the mean of a vector.

In [ ]:
mean(myvector)

The `length` function gives the number of elements in a vector.

In [ ]:
length(myvector)

The *table* function takes a vector, finds all the possible unique values in the vector, and calculates the number of times that each value appears in the vector.

In [ ]:
class_majors = c("Psychology","Public Health","Biology","Biology","Computer Science","Biology")
table(class_majors)

Functions can take multiple arguments. The *log(x,y)* function takes the base *y* log of *x*.

In [ ]:
log(16,2)

In [ ]:
log(27,3)

### Getting help on functions
A lot of functions that come with R or R packages provides *inline help*. This allows you to see how to call the functions and what the functions do.

Usually you can just type ? before the function name.

Try getting help for the `log` function.

In [ ]:
?log

And for the `mean` function.

In [ ]:
?mean

## Single cell Ciona dataset - spatial dynamics

### Loading the R packages and the spatial dataset

Many people have written functions for doing analysis in R. The functions are put in R *packages* and *libraries* that must be installed or loaded so the functions can be used.

We use the `library` function here to load the `Seurat` package that provides many functions to analyze single-cell RNA-seq data, and the `RColorBrewer` package that tells R what colors we would use.  The `source` function loads the convenient functions that we use in this lab.

In [ ]:
library(Seurat)
library(RColorBrewer)
source('../src/functions.R')

We can now load the `hpf20` object from the R file *spatial.Robj*, which contains the analyzed information about the transcriptomes of cells collected at hpf20. Then load the *temporal.RObj* file that contains the trajectories of all the cell types.

In [ ]:
load('../data/external/spatial.Robj')
load('../data/external/temporal.Robj')

After loading, you can call the `ls` function to check what variables are currently in the memory.

In [ ]:
ls()

## Reading the marker genes

We have a few text files that specify the differential expression of the marker genes in each cell type. Here we read these files into R and filter to keep the ones that show the strongest differential expression

In [ ]:
heartGene = rownames(subset(read.table("../data/external/panHP_20.markers.txt"), power>0.5 & avg_diff>1))
asmGene = rownames(subset(read.table("../data/external/ASM_20.markers.txt"), power>0.5 & avg_diff>1))
shpSpecific = rownames(subset(read.table("../data/external/SHPspecific_20.markers.txt"),power>0.5&avg_diff>1))
fhpSpecific = rownames(subset(read.table("../data/external/FHPspecific_20.markers.txt"),power>0.5&avg_diff>1))    

**Exercise**: How many genes are in each of the vectors *heartGene*, *asmGene*, *shpSpecific* and *fhpSpecific*?  Try to calculate this in the cell below.

For the purpose of this lab, we will use up to 15 marker genes for each cell type.  We will also define a *AllGene* vector that holds all the (subset of) marker genes.

In [ ]:
ASM = asmGene[1:15]
Heart = heartGene[1:15]
FHP_Specific = fhpSpecific[1:15]
SHP_Specific = shpSpecific
AllGene = c(heartGene[1:15], fhpSpecific[1:15], shpSpecific,asmGene[1:15])

### Getting information from the hpf20 object

In [ ]:
hpf20

The *hpf20* object contains many datasets related to the analysis of hpf20 dataset.  We can get the raw sequencing read counts for each gene in each cell in the *raw.data* variable.

In [ ]:
hpf20@raw.data

### Plotting: tSNE plots, violin plots and heatmaps

The *hpf20* object contains information about a pre-computed tSNE plot.  We can use the `TSNEPlot` function to draw it 

In [ ]:
TSNEPlot(hpf20)

The `TSNEPlot` function can take many arguments that allow customizing the tSNE plot.

In [ ]:
?TSNEPlot

We can add the cell identity labels and change the colors of the inferred cell identity.

In [ ]:
TSNEPlot(hpf20,do.label=TRUE,colors.use = c("red","orange","lightblue","blue"),pt.size = 1.5)

Using the `FetchData` function, we can take a look at the identity and the transformed dimsion of each cell.

In [ ]:
FetchData(hpf20,c("ident","tSNE_1","tSNE_2"))

`FetchData` can also get us the expression levels of EBF genes (ASM marker) in each cell?

In [ ]:
FetchData(hpf20,'EBF1/2/3/4')

We can overlay the expression of the EBF genes onto the tSNE plot by the `FeaturePlot` function.  This is presented in Fig. 1b.

In [ ]:
FeaturePlot(hpf20,features.plot = 'EBF1/2/3/4', pt.size = 2)

**Exercise**: Draw this for the heart maker gene 'GATA4/5/6'.

`FeaturePlot` can also take a vector of genes to draw all of them in multiple panels.  Recall the *ASM* variable is a vector of (a subset of) ASM makers.

In [ ]:
FeaturePlot(hpf20,features.plot = ASM,pt.size = 1.5)

What do the plots look like if you overlay the heart maker expression?

In [ ]:
FeaturePlot(hpf20,features.plot = Heart,pt.size = 1.5)

Violin plots show the distribution of the markers in each cell type.  We first increase the size of the plot so each panel is visible (default is 7 by 7).

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 10)
VlnPlot(hpf20, Heart)

We can show the expression of all the markers across all the cell types in a heatmap.  Note we use all the marker genes stored in the *AllGene* variable.  This is already quite similar to the heatmap in Fig. 1c. We just need to change the color mapping and change the row/column orders.

In [ ]:
DoHeatmap(hpf20,genes.use=AllGene, order.by.ident = TRUE,slim.col.label = TRUE, draw.line = TRUE,mar=c(8,8))

The code below uses more options in the `DoHeatmap` function, getting pretty close to the one presented in Fig. 1c.

In [ ]:
DoHeatmap(hpf20,genes.use = AllGene,order.by.ident = T,
          slim.col.label = T,draw.line = T,key.title = "Expression Level Scale",
          key.xlab = "", key.ylab = "",
          RowSideColors = c(rep("pink",15),rep("red",15),rep("orange",4),rep("blue",15)), 
          mar=c(8,8),col=col,sepcolor="black",sepwidth = c(1,0.2),
          srtCol=0,cex.col = 1,cexRow=1,keysize=1,density.info=c("none"))

### Single cell Ciona dataset - temporal dynamics

The *asm.test*, *shp.test*, and *fhp.test* objects that we loaded previously contain pre-computed trajectories for the ASM, SHP and FHP clusters, respectively.  The *ps.asm*, *ps.shp* and *ps.fhp* objects contain the pre-computed regulatory state transition times for the correspoding trajectories.

Here we plot the expression of the SHP marker 'DACH1/2' along the SHP pseudotime trajectory, mark the activation pseudotime (purple vertical line), and the transition between regulatory states. This reproduceds Fig. 2f middle panel.

We first reset the figure size back to the original 7 by 7.

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7)
genePlot.pseudo(fhp.test,gene='GATA4/5/6',
                col.use = c("green","pink1","red1","red4"),
                do.spline = T,
                name.x = "FHP Trajectory",
                cex.use = 0.8,cex.lab=1,do.logit = T)
abline(v=ps.fhp,lwd=2,lty=2)

We can also plot all the FHP marker genes.

In [ ]:
genes.plot.pseudo(fhp.test,genes.use = FHP_Specific, lwd=1.5, name.x = "FHP")

## Breakout Room Exercises

1. Show the expression of FHP marker 'MMP21' and SHP marker 'DACH1/2' the tSNE plot.  Then reproduce the violin plots in Fig. 1d for 'MMP21' and 'DACH1/2'.  What are the pro's and con's of each representation?


2. How is the ASM marker 'EBF1/2/3/4' expressed along the ASM pseudotime trajectory (i.e., reproduce the EBF panel in Fig. 3a)?  Along the FHP trajectory?
  * The colors use in that figure is `c("green","yellow","lightblue","blue1","blue4")`.


3. Create a heatmap of the ASM trajectory, i.e., using the `DoHeatmap` function with the *asm.test* object and *ASM* gene list, and compare to the heatmap in Fig. 3b.